<a href="https://colab.research.google.com/github/nipdep/SpCup-22-Init/blob/team1/param_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
%cd /gdrive/MyDrive/SP Cup 2022/SpCup-22-Init/src/sbcnn

/gdrive/MyDrive/SP Cup 2022/SpCup-22-Init/src/sbcnn


In [ ]:
! pip install neptune-client &> /dev/null

In [ ]:
!pip install audiomentations &> /dev/null

In [ ]:
!pip install pedalboard &> /dev/null

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPool2D, Input
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L2
from tensorflow.keras.callbacks import Callback
import warnings
import neptune.new as neptune
from dataloader_v1 import SpectDataset
import numpy as np
from sklearn.metrics import classification_report
import itertools
from getpass import getpass

warnings.filterwarnings('ignore')

In [ ]:
api_token = getpass('Enter your private Neptune API token: ')

Enter your private Neptune API token: ··········


In [ ]:
workspace = 'chalani.ekanayake212' 
project = 'SBCNN-SPCUP'
project_name = workspace + '/' + project

In [ ]:
run = neptune.init(project=project_name, api_token=api_token)

https://app.neptune.ai/chalani.ekanayake212/SBCNN-SPCUP/e/SBCNNSPC-7
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [ ]:
def SBCNN_Model(input_shape, num_labels):
    model = Sequential()
    model.add(Input(input_shape))
    model.add(Conv2D(24, 5, padding='same'))
    model.add(MaxPool2D(strides=(4, 2)))
    model.add(Activation('relu'))

    model.add(Conv2D(48, 5, padding='same'))
    model.add(MaxPool2D(strides=(4, 2)))
    model.add(Activation('relu'))

    model.add(Conv2D(48, 5, padding='valid'))
    model.add(Activation('relu'))

    model.add(Flatten())

    model.add(Dropout(0.5))
    model.add(Dense(64, kernel_regularizer=L2(0.001)))
    model.add(Activation('relu'))
    
    model.add(Dropout(0.5))
    model.add(Dense(num_labels, kernel_regularizer=L2(0.001)))
    model.add(Activation('softmax'))

    return model

In [ ]:
def run_model(optimizer, n_iter):
        
    DATA_PATH = '/gdrive/MyDrive/SP Cup 2022/spcup_2022_training_part1'
    Labeled_dir = '/gdrive/MyDrive/SP Cup 2022/spcup_2022_training_part1/labels.csv'

    dataloader = SpectDataset(add_noise=True, reverb=True)
    train_ds, val_ds = dataloader.call(DATA_PATH, Labeled_dir)

    for spectrogram, _ in train_ds.take(1):
        input_shape = spectrogram.shape

    model = SBCNN_Model(input_shape[1:], 6)

    class PrintLRCallback(Callback):
      def on_epoch_end(self, epoch, logs={}):
        print("\nSBCNN with learning rate {} \n".format(model.optimizer._decayed_lr('float32').numpy()))
        for log_name, log_value in logs.items():
            run["epoch/{}".format(log_name)].log(log_value)
            run["epoch/{}".format('Learning Rate')].log(model.optimizer._decayed_lr('float32').numpy())

    model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=['accuracy']
    )

    EPOCHS = n_iter
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS,
        verbose=2,
        callbacks=[PrintLRCallback()]
    # callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2),
    )

    res_sparse = model.predict(val_ds)
    res_np = np.argmax(res_sparse, axis=1)
    pred_labels = list(res_np)
    true_labels = []
    for _, label in val_ds.take(1000):
        true_labels.extend(list(label.numpy()))

   # class_names = [f'class_{i}' for i in range(6)]
   # print(classification_report(true_labels, pred_labels, target_names=class_names))

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
              initial_learning_rate=0.1,
              decay_steps=(1000), 
              decay_rate=0.5)

sgd = SGD(learning_rate = lr_schedule)
adm = Adam(learning_rate=lr_schedule)

n_iter = 20

optimizer = sgd

# log params
run["parameters/epochs"] = n_iter

run_model(optimizer, n_iter)

Epoch 1/20

SBCNN with learning rate 0.09170039743185043 

125/125 - 83s - loss: 1.3938 - accuracy: 0.4820 - val_loss: 0.8355 - val_accuracy: 0.7140 - 83s/epoch - 661ms/step
Epoch 2/20

SBCNN with learning rate 0.08408963680267334 

125/125 - 3s - loss: 0.7882 - accuracy: 0.7235 - val_loss: 0.6493 - val_accuracy: 0.7760 - 3s/epoch - 23ms/step
Epoch 3/20

SBCNN with learning rate 0.07711053639650345 

125/125 - 3s - loss: 0.6566 - accuracy: 0.7772 - val_loss: 0.4880 - val_accuracy: 0.8620 - 3s/epoch - 22ms/step
Epoch 4/20

SBCNN with learning rate 0.0707106739282608 

125/125 - 3s - loss: 0.5473 - accuracy: 0.8160 - val_loss: 0.4556 - val_accuracy: 0.8700 - 3s/epoch - 22ms/step
Epoch 5/20

SBCNN with learning rate 0.06484197825193405 

125/125 - 3s - loss: 0.4823 - accuracy: 0.8555 - val_loss: 0.4016 - val_accuracy: 0.8990 - 3s/epoch - 22ms/step
Epoch 6/20

SBCNN with learning rate 0.0594603531062603 

125/125 - 3s - loss: 0.4273 - accuracy: 0.8742 - val_loss: 0.3846 - val_accuracy: 0.9

In [ ]:
run.stop()

Shutting down background jobs, please wait a moment...
Done!


Waiting for the remaining 15 operations to synchronize with Neptune. Do not kill this process.


All 15 operations synced, thanks for waiting!
